# N-Gram Strategy

Break my sequences down into n-grams. Then;
1. merge/map the protein-level sequence features into the n-gram row
2. merge/map the amino acid-level features into the n-gram row
3. build the final feature matrix
4. do some feature selection, dimension reduction
5. train the model
6. evaluate

In [ ]:
import sqlite3
import nltk
import pandas as pd
import os

In [ ]:
conn = sqlite3.connect("human_protein.db")
prot_feat = pd.read_sql('select * from protein_features',con=conn)
amy_df = pd.read_sql('select * from amyloid', con=conn)
conn.close()
prot_feat.head()

In [ ]:
def nGrammer(protein, seq, n):
    return [[protein, i+1]+list(x) for i, x in enumerate(nltk.ngrams(seq, n))]

def dfNGrammer(protein_df, num_grams):
    gram_cols = ["protein","gram_num"]+["gram_"+str(i+1) for i in range(num_grams)]
    seq_grams = []
    seq_grams.extend([val for sublist in protein_df.apply(lambda row: nGrammer(row['protein'], row['sequence'], num_grams),axis=1) for val in sublist])
    gram_df = pd.DataFrame(seq_grams, columns=gram_cols)
    return gram_df

In [ ]:
#ngram_df = dfNGrammer(prot_feat,5)